In [47]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from modules import *

from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt #형태소 분석
from nltk import bigrams
from nltk.util import ngrams

from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel # 토픽 개수 검증
from gensim import corpora
from gensim.models import TfidfModel

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')


In [48]:
data = pd.read_csv("술 리뷰(복순도가).csv", encoding='utf-8-sig')
data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
data.columns = ['date', 'star','doc','name']
# del data['Unnamed: 0']
data

,date,star,doc,name
0,2022-10-31,5,지인으로부터 알게된 복순도가 막걸리처음 먹는순간 너무 맛있어서 깜짝놀랐어요원래 술을...,복순도가
1,2022-11-17,5,그냥 차게 마셔도 군내나 잔여물 찌꺼기 느낌이 거의 없이 깔끔담백하고 샴페인처럼 톡...,복순도가
2,2022-10-19,5,지인이 비싼데 고급지고 맛있는 막걸리가있다고 해서하도 궁금하여 한번 시켜봤어요배송도...,복순도가
3,2022-10-28,3,본인은 맛있었는데 아버지는 너무 삭힌거같다는 평이네요 새콤함이 좀 있긴함근데 막걸리...,복순도가
4,2022-10-13,5,복순도가 맛있다고 한참전에 소문만 들었지 먹어본것은 올해가 처음이네요 평소에도 달달...,복순도가
...,...,...,...,...
19983,2022-11-28,5,벌써 두번째 주문입니다 김장때 수육이랑 같이 먹었는데 너무 맛있었고 어른들이 특히 ...,복순도가
19984,2022-11-28,5,술 한잔도 못드시는 어머니도 너무 맛있게 잘 드시네요,복순도가
19985,2022-11-28,5,역시 최고에요최고 최고,복순도가
19986,2022-11-28,5,맛ㅇㅅ어요 집에서 싸게 주문해서 먹어서 맛있어요ㅔ,복순도가


In [49]:
data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
data.doc

0        지인으로부터 알게된 복순도가 막걸리처음 먹는순간 너무 맛있어서 깜짝놀랐어요원래 술을...
1        그냥 차게 마셔도 군내나 잔여물 찌꺼기 느낌이 거의 없이 깔끔담백하고 샴페인처럼 톡...
2        지인이 비싼데 고급지고 맛있는 막걸리가있다고 해서하도 궁금하여 한번 시켜봤어요배송도...
3        본인은 맛있었는데 아버지는 너무 삭힌거같다는 평이네요 새콤함이 좀 있긴함근데 막걸리...
4        복순도가 맛있다고 한참전에 소문만 들었지 먹어본것은 올해가 처음이네요 평소에도 달달...
                               ...                        
19983    벌써 두번째 주문입니다 김장때 수육이랑 같이 먹었는데 너무 맛있었고 어른들이 특히 ...
19984                        술 한잔도 못드시는 어머니도 너무 맛있게 잘 드시네요
19985                                         역시 최고에요최고 최고
19986                           맛 어요 집에서 싸게 주문해서 먹어서 맛있어요 
19987                        와 인생막걸리입니다 이제 다른거 못먹어요복순도가 최고
Name: doc, Length: 19988, dtype: object

In [50]:
# 검토 후 추가 삭제 문구 적용
data.dropna(inplace=True)

In [51]:
data[data.doc.isnull()]

,date,star,doc,name


In [52]:
# data = data[data.doc.str.contains('맛|향|달|넘김|부드|단|도수|냄세')]
# data.reset_index(drop=True, inplace=True)

In [53]:
len(data)

19987

In [54]:
# #!/usr/bin/env python
# # coding: utf-8

# from nltk.corpus import stopwords
# import nltk
# from nltk.stem import WordNetLemmatizer, SnowballStemmer
# from nltk.stem.porter import *
# from nltk import word_tokenize
# from konlpy.tag import Okt #형태소 분석
# from nltk import bigrams
# from nltk.util import ngrams

# from gensim.models.ldamodel import LdaModel
# import matplotlib.pyplot as plt
# from gensim.models.coherencemodel import CoherenceModel # 토픽 개수 검증
# from tqdm import tqdm
# # from tqdm.notebook import tqdm
# from gensim import corpora
# from gensim.models import TfidfModel

# import warnings
# warnings.filterwarnings(action='ignore')

# from modules import *

In [55]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','쓰리다 도어즈','일요일','휴무','수표','층','틀다','차차','드므','맛집','술집','길','가가'
                 '층','개다 성','사진','파다','동영상','아스 론','자다','는','희다 스토리','한','씨', "것","거","게","데",
                 "이다","건","고","되다","되어다","걸","기", "시","네","듯","랍니","중이","얘","스","도도", "나","수","개",
                 "내","기","제","저","인","있다","이렇다", "그렇다","번","위","팅","분","인","링","란","포","두", "진짜",
                 "하다" ,"이다" ,"가다", "이제" ,"들다", "먹다", "보다"]) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]
             
    return words


In [56]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer = lambda x: okt_pos_tagging2(x))
bow_vect = vect.fit_transform(data['doc'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [57]:
bow_vect.shape

(19987, 7029)

In [58]:
word_count_dict = dict(zip(word_list, count_list))
word_count_dict


{'가가': 61,
 '가감': 1,
 '가걱': 1,
 '가게': 21,
 '가격': 812,
 '가격떄문': 1,
 '가계': 1,
 '가공': 1,
 '가구': 2,
 '가기': 6,
 '가까워지다': 1,
 '가까이': 4,
 '가깝다': 23,
 '가끔': 125,
 '가나': 2,
 '가난': 1,
 '가난하다': 1,
 '가내': 1,
 '가늘다': 1,
 '가능': 1,
 '가능성': 2,
 '가능하다': 24,
 '가도': 8,
 '가두': 1,
 '가두다': 1,
 '가득': 36,
 '가득가득': 1,
 '가득하다': 54,
 '가라': 12,
 '가라앉다': 3,
 '가라앉히다': 3,
 '가로': 35,
 '가루': 1,
 '가르다': 92,
 '가리다': 6,
 '가리비': 1,
 '가마': 1,
 '가막': 1,
 '가만': 39,
 '가만히': 1,
 '가면': 9,
 '가물': 4,
 '가미': 6,
 '가바': 1,
 '가방': 8,
 '가버리다': 4,
 '가벼워지다': 1,
 '가볍다': 185,
 '가보': 1,
 '가보다': 23,
 '가복': 6,
 '가봉': 1,
 '가분': 1,
 '가뿐하다': 1,
 '가사': 3,
 '가성': 25,
 '가세': 1,
 '가쇼': 1,
 '가수': 1,
 '가스': 16,
 '가시': 2,
 '가시다': 5,
 '가신': 2,
 '가야': 3,
 '가오니': 1,
 '가온': 1,
 '가요': 27,
 '가운데': 2,
 '가은': 1,
 '가을': 1,
 '가의': 27,
 '가이거': 2,
 '가이드': 1,
 '가이드북': 1,
 '가인': 1,
 '가장': 63,
 '가절': 1,
 '가정': 3,
 '가제': 2,
 '가져가다': 50,
 '가져오다': 5,
 '가족': 551,
 '가중': 1,
 '가즈': 1,
 '가지': 29,
 '가지다': 8,
 '가집': 2,
 '가차': 1,
 '가치': 15,
 '가타': 2,
 '가평': 1,
 '가하다': 5,
 '가혹

In [59]:
def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words

In [60]:
tokens = [] 

for i in data['doc']:
    token = okt_pos_tagging(i)
    tokens.append(token)

tokens

[['지인',
  '알',
  '복',
  '순도',
  '막걸리',
  '처음',
  '순간',
  '너무',
  '맛있다',
  '깜짝',
  '원래',
  '술',
  '좋아하다',
  '않다',
  '사람',
  '수시로',
  '냉장고',
  '꺼내다',
  '말다',
  '맛',
  '달',
  '크다',
  '시큰하다',
  '샴페인',
  '탄산',
  '자잘하다',
  '탄산',
  '목',
  '넘김',
  '좋다',
  '요술',
  '않다',
  '맛',
  '은근',
  '히',
  '알코올',
  '올라오다',
  '암튼',
  '너무나',
  '맛있다',
  '고급',
  '진',
  '막걸리',
  '굳이',
  '단점',
  '말다',
  '처음',
  '오픈',
  '걸치다',
  '탄산',
  '빼다',
  '따다',
  '정말',
  '샴페인',
  '터지다',
  '여기저기',
  '난리',
  '나다',
  '아무튼',
  '너무나',
  '찰떡',
  '술',
  '주변',
  '지인',
  '계속',
  '홍보'],
 ['그냥',
  '차다',
  '마시다',
  '군내',
  '잔',
  '여물',
  '찌꺼기',
  '느낌',
  '없이',
  '깔다',
  '끄다',
  '담백하다',
  '샴페인',
  '톡',
  '쏘다',
  '맛',
  '상큼',
  '함',
  '전통',
  '샴페인',
  '명명하다',
  '좋다',
  '맛있다',
  '요전',
  '병',
  '비우다',
  '냉동',
  '보관',
  '꺼내다',
  '살얼음',
  '막걸리',
  '마시다',
  '그야말로',
  '최고',
  '술',
  '욕심',
  '나서다',
  '병',
  '얼리다',
  '입구',
  '가늘다',
  '기다',
  '마개',
  '따다',
  '김',
  '빠지다',
  '입구',
  '상태',
  '좁다',
  '씨름',
  '다가',
  '아깝다',
  '컵',
  '분량',
  '낭비하다',
  

In [61]:
from collections import Counter

# 토큰화된 단어 하나의 리스트로 만듦
all_tokens = []
for i in tokens:
    all_tokens = all_tokens + i

counted = Counter(all_tokens)

In [62]:
at_df = pd.DataFrame(columns=['mentioned'])

# 단어별 언급량 비율화 
for i,j in zip(counted.keys(),counted.values()):
    at_df.loc[i] = j/len(counted)

In [63]:
at_df = at_df.sort_values(by='mentioned',ascending=False)
at_df

,mentioned
맛있다,1.244416
막걸리,0.813772
맛,0.812349
좋다,0.803101
너무,0.628966
...,...
로나,0.000142
야합,0.000142
호풀,0.000142
강물,0.000142


In [64]:
from gensim.models import Word2Vec

w2v = Word2Vec(tokens, vector_size= 300, window=2, min_count=3, epochs=20)
word_vectors = w2v.wv
vocabs = word_vectors.key_to_index
word_vectors_list = [word_vectors[v] for v in vocabs]

In [65]:
# 단어 추출에서 탈락시킬 단어
stopwords2 = ['나다', '안나', '이나', '기대하다', '나서다', '끝','싹','이나고','잇다','기대다','생각','기대',
            '랄','날','끄다','편이','끼','감','감다','기도','함','테','넘어가다','비다','느껴지다','진하다',
            '차다','약하다','도인','올라오다','일반','아니다','나다','넘어가다','비다','살짝','높다','도수','뒤','마지막',
            '넘기다','없다','요','도','더','집','않다','많이','느낌','만들다','먹기','자체','해','덜','들억다','히','도르다',
            '막히다','한식','김','개인','젤','상당하다','인상','확','입안'
            ,'안달','미가','풍','콤함','알코올','살아나다','면서','제게','인위','강하','긋다','콤해']

In [66]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')

{5633: '지인', 4017: '알', 2788: '복', 3571: '순도', 1985: '막걸리', 5842: '처음', 3569: '순간', 1035: '너무', 2098...


In [68]:
from sklearn.model_selection import train_test_split

x = word_vectors_list
y = data['star']

In [69]:
(x.shape, y.shape)

AttributeError: 'list' object has no attribute 'shape'

In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LinearRegression()
lr.fit(x, y)

# predict in test set
y_pred = lr.predict(x)

ValueError: Found input variables with inconsistent numbers of samples: [2823, 19987]

In [ ]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = False)

In [ ]:
weight_df = pd.DataFrame(columns=['weight'])
# 단어별 가중치 확인 
for coef in coef_neg_index[:]:
    weight_df.loc[invert_index_vectorizer[coef[1]]] = coef[0]
    print(invert_index_vectorizer[coef[1]],coef[0])

In [ ]:
a = w2v.wv.most_similar(positive=['목','넘김'])
a = [word for word, tag in a if word not in stopwords2]
a2 = pd.DataFrame(columns=['언급비율', '가중치'])

for i in a: 
    a2.loc[i] = [at_df.loc[i].item(),weight_df.loc[i].item()]
a2

In [ ]:
print('accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('precision: %.4f' % precision_score(y_test, y_pred))
print('recall: %.4f' % recall_score(y_test, y_pred))
print('F1: %.4f' % f1_score(y_test, y_pred))


accuracy: 0.9570
precision: 0.9591
recall: 0.9975
F1: 0.9780


In [ ]:
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])

맛있다 5.619819103741862
최고 3.501886725561354
꼼꼼하다 2.8828111090772905
좋다 2.816866974316315
잇다 2.7625418621958184
안전하다 2.3969567828000233
만족하다 2.2942696891367476
자주 2.168771627405244
부드럽다 2.140136901536306
역시 2.0594936774047805
늘 2.0117746731093726
부담 1.9295527154683214
믿다 1.9250481797242294
재다 1.903197570198565
항상 1.8108860641760651
고급 1.8045112952204814
아주 1.770342682122279
굳다 1.6550696350190612
매력 1.6218748490269392
예쁘다 1.6137895553906465


In [ ]:
for coef in coef_neg_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])

별로 -3.64484485468993
아니다 -3.480255023937562
그냥 -3.460422143582286
맛없다 -3.32792823866652
셔 -3.144755607812021
실망 -3.043034095210721
신맛 -2.9908113278020303
비추다 -2.8516068139123663
이하 -2.704936050118673
시큼하다 -2.679910958712632
별루 -2.675329540419246
루 -2.5862393375554027
다시다 -2.56297127564243
밍밍 -2.442430637435147
달다 -2.4037526970278544
이상하다 -2.3629506444469364
건지다 -2.2673343413881724
보통 -2.2470344454605695
식초 -2.2101673014688332
불량 -2.1394335020294584


In [ ]:
for coef in coef_neg_index[:]:
    print(invert_index_vectorizer[coef[1]], coef[0])

별로 -3.64484485468993
아니다 -3.480255023937562
그냥 -3.460422143582286
맛없다 -3.32792823866652
셔 -3.144755607812021
실망 -3.043034095210721
신맛 -2.9908113278020303
비추다 -2.8516068139123663
이하 -2.704936050118673
시큼하다 -2.679910958712632
별루 -2.675329540419246
루 -2.5862393375554027
다시다 -2.56297127564243
밍밍 -2.442430637435147
달다 -2.4037526970278544
이상하다 -2.3629506444469364
건지다 -2.2673343413881724
보통 -2.2470344454605695
식초 -2.2101673014688332
불량 -2.1394335020294584
입맛 -2.138068907140942
기대하다 -2.043734723456683
맞다 -2.032811560193659
느리다 -2.0208706228243525
실망하다 -2.014372194385718
확인 -1.99976510445999
차이 -1.9988960523829418
터지다 -1.914196950053748
물탄 -1.9042570264927143
애매하다 -1.8701601754025545
돈 -1.8652570665734969
그다지 -1.8633545794380233
비싸다 -1.8565524539758618
아쉽다 -1.8412227456083186
평범하다 -1.8358725029344503
도저히 -1.8342107822530223
버리다 -1.8225082104015764
싱겁다 -1.8198861479170618
밋밋하다 -1.7947769716961353
불다 -1.7917378885846982
강하다 -1.788407809505488
그나마 -1.781835737253066
변하다 -1.762442916322569
모르다 -1.7